In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from datetime import datetime
import os

# === 1. Cấu hình Chrome an toàn ===
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1400,900")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(30)
driver.implicitly_wait(5)

# === 2. Đọc file cũ để tránh cào trùng ===
old_file = "dantri_all_rss.csv"
old_links = set()

if os.path.exists(old_file):
    old_data = pd.read_csv(old_file)
    if "link" in old_data.columns:
        old_links = set(old_data["link"].dropna().tolist())

print(f" Đã đọc {len(old_links)} link cũ để tránh trùng.")

# === 3. Danh sách chuyên mục cần cào (mở rộng) ===
categories = {
    "Thời sự": "https://dantri.com.vn/thoi-su.htm",
    "Kinh doanh": "https://dantri.com.vn/kinh-doanh.htm",
    "Thế giới": "https://dantri.com.vn/the-gioi.htm",
    "Văn hóa": "https://dantri.com.vn/van-hoa.htm",
    "Giải trí": "https://dantri.com.vn/giai-tri.htm",
    "Thể thao": "https://dantri.com.vn/the-thao.htm",
    "Pháp luật": "https://dantri.com.vn/phap-luat.htm",
    "Sức khỏe": "https://dantri.com.vn/suc-khoe.htm",
    "Giáo dục": "https://dantri.com.vn/giao-duc.htm",
    "Du lịch": "https://dantri.com.vn/du-lich.htm",
    "Infographic": "https://dantri.com.vn/infographic.htm",
    "Ô tô - Xe máy": "https://dantri.com.vn/o-to-xe-may.htm",
    "Bạn đọc": "https://dantri.com.vn/ban-doc.htm",
    "Tâm điểm": "https://dantri.com.vn/tam-diem.htm",
    "Nhịp sống trẻ": "https://dantri.com.vn/nhip-song-tre.htm",
    "Tình yêu - Giới tính": "https://dantri.com.vn/tinh-yeu-gioi-tinh.htm",
    "Sức mạnh số": "https://dantri.com.vn/suc-manh-so.htm",
    "Chuyện lạ": "https://dantri.com.vn/chuyen-la.htm",
    "Nhà đất": "https://dantri.com.vn/nha-dat.htm",
    "Tấm lòng nhân ái": "https://dantri.com.vn/tam-long-nhan-ai.htm",
    "Việc làm": "https://dantri.com.vn/lao-dong-viec-lam.htm",
    "Khám phá": "https://dantri.com.vn/kham-pha.htm",
    "Blog": "https://dantri.com.vn/blog.htm",
    "Xã hội": "https://dantri.com.vn/xa-hoi.htm"
}

# === 4. Cấu hình giới hạn cuộn & trang ===
scroll_limit = 10   # số lần cuộn mỗi trang
page_limit = 15     # số trang muốn duyệt tối đa

# === 5. Bắt đầu cào ===
all_rows = []
crawl_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for category, url in categories.items():
    print(f"\n Đang cào chuyên mục: {category}")
    for page in range(1, page_limit + 1):
        page_url = url.replace(".htm", f"/trang-{page}.htm")
        driver.get(page_url)
        time.sleep(2)

        # Cuộn để tải thêm nội dung
        for _ in range(scroll_limit):
            driver.execute_script("window.scrollBy(0, document.body.scrollHeight);")
            time.sleep(1.2)

        # Tìm bài viết
        articles = driver.find_elements(By.CSS_SELECTOR, "article a[href*='dantri.com.vn']")
        print(f" Trang {page}: tìm thấy {len(articles)} link.")

        for a in articles:
            title = a.text.strip()
            link = a.get_attribute("href")

            # Bỏ qua nếu trùng hoặc không có tiêu đề
            if not title or link in old_links:
                continue

            all_rows.append({
                "category": category,
                "title": title,
                "link": link,
                "crawl_date": crawl_date
            })
            old_links.add(link)

        # Nếu trang không còn bài mới → dừng sớm
        if len(articles) < 5:
            break

# === 6. Lưu file CSV mới ===
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_file = f"dantri_new_unique_{timestamp}.csv"
pd.DataFrame(all_rows).to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"\n Hoàn tất! Lưu {len(all_rows)} bài mới vào: {output_file}")
driver.quit()


 Đã đọc 1300 link cũ để tránh trùng.

 Đang cào chuyên mục: Thời sự
 Trang 1: tìm thấy 129 link.
 Trang 2: tìm thấy 125 link.
 Trang 3: tìm thấy 125 link.
 Trang 4: tìm thấy 125 link.
 Trang 5: tìm thấy 125 link.
 Trang 6: tìm thấy 125 link.
 Trang 7: tìm thấy 125 link.
 Trang 8: tìm thấy 125 link.
 Trang 9: tìm thấy 125 link.
 Trang 10: tìm thấy 125 link.
 Trang 11: tìm thấy 125 link.
 Trang 12: tìm thấy 108 link.
 Trang 13: tìm thấy 108 link.
 Trang 14: tìm thấy 108 link.
 Trang 15: tìm thấy 108 link.

 Đang cào chuyên mục: Kinh doanh
 Trang 1: tìm thấy 128 link.
 Trang 2: tìm thấy 124 link.
 Trang 3: tìm thấy 108 link.
 Trang 4: tìm thấy 108 link.
 Trang 5: tìm thấy 108 link.
 Trang 6: tìm thấy 108 link.
 Trang 7: tìm thấy 108 link.
 Trang 8: tìm thấy 108 link.
 Trang 9: tìm thấy 108 link.
 Trang 10: tìm thấy 108 link.
 Trang 11: tìm thấy 108 link.
 Trang 12: tìm thấy 108 link.
 Trang 13: tìm thấy 108 link.
 Trang 14: tìm thấy 108 link.
 Trang 15: tìm thấy 108 link.

 Đang cào chuyê